# Syft-Client Interactive Tutorial

Welcome to the syft-client tutorial! This notebook demonstrates all the core features of syft-client for secure, decentralized file sharing using Google Drive.

## What You'll Learn

1. **Authentication** - How to login and manage credentials
2. **SyftBox Setup** - Creating your secure communication hub
3. **Friend Management** - Adding friends and managing connections
4. **Secure Messaging** - Using SyftMessage for file transfer
5. **Two-User Workflow** - Complete bidirectional setup
6. **Best Practices** - Tips for production use

Let's get started!

# Clean installation - ensures you get the latest version
# This will uninstall any existing version and install the latest

import subprocess
import sys

# Uninstall existing version if present
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "syft-client"], 
               capture_output=True, text=True)

# Install latest version from PyPI
subprocess.run([sys.executable, "-m", "pip", "install", "syft-client"], 
               capture_output=True, text=True)

# Alternative installation options (uncomment as needed):
# 
# For specific version:
# !pip install syft-client==0.1.5
#
# For development from GitHub:
# !pip install git+https://github.com/OpenMined/syft-client.git
#
# For local development:
# !pip install -e /path/to/syft-client

# Import the library
import syft_client as sc
from pathlib import Path
import tempfile
import json

print(f"✅ Syft-Client version: {sc.__version__}")
print(f"📦 Installation location: {sc.__file__}")

In [ ]:
# Uncomment for Google Colab or fresh environment
# !pip install syft-client

# Import the library
import syft_client as sc
from pathlib import Path
import tempfile
import json

print(f"Syft-Client version: {sc.__version__}")

## 2. Authentication

### First-Time Setup

If this is your first time, you'll need Google Drive API credentials. The `login()` function will guide you through the process.

In [ ]:
# Replace with your email
YOUR_EMAIL = "your_email@gmail.com"

# First login - will open browser for authentication
client = sc.login(YOUR_EMAIL)

# Check authentication status
print(f"\nAuthenticated: {client.authenticated}")
print(f"Logged in as: {client.my_email}")

### Managing Multiple Accounts

Syft-client can manage multiple Google accounts. Let's see how:

In [ ]:
# List all accounts with saved credentials
accounts = sc.list_accounts()
print("Saved accounts:")
for account in accounts:
    print(f"  - {account}")

# You can login to any saved account without re-authenticating
if accounts:
    # Login to first saved account
    client = sc.login(accounts[0], verbose=False)
    print(f"\nSwitched to: {client.my_email}")

## 3. SyftBox Setup

A SyftBox is your secure communication hub in Google Drive. Let's create one:

In [ ]:
# Create or reset your SyftBox
# Warning: This will delete any existing SyftBox!
folder_id = client.reset_syftbox()

print(f"\nSyftBox created with ID: {folder_id}")
print("\nYour SyftBox is now ready for secure communication!")

### Understanding the Client Object

The client object provides a nice summary of your setup:

In [ ]:
# Display client information
client

## 4. Friend Management

To communicate securely, you need to set up friend connections. Let's explore how this works:

In [ ]:
# Check current friends
print("Current friends:")
if client.friends:
    for friend in client.friends:
        print(f"  - {friend}")
else:
    print("  No friends yet")

# Check friend requests (people who added you)
print("\nFriend requests:")
if client.friend_requests:
    for request in client.friend_requests:
        print(f"  - {request}")
else:
    print("  No pending requests")

### Adding a Friend

To add a friend, you need their email address. They'll need to add you back to complete the bidirectional channel.

In [ ]:
# Example: Add a friend (replace with actual email)
FRIEND_EMAIL = "friend@example.com"  # Change this!

# Uncomment to actually add a friend
# success = client.add_friend(FRIEND_EMAIL)
# if success:
#     print(f"Successfully added {FRIEND_EMAIL} as a friend!")
#     print("They'll need to add you back to complete the connection.")
# else:
#     print(f"Failed to add {FRIEND_EMAIL}")

print("To add a friend, uncomment the code above and use a real email.")

## 5. Working with SyftMessage

SyftMessage provides secure, atomic file transfer with permissions. Let's create a message:

In [ ]:
# Create a temporary directory for our demo
with tempfile.TemporaryDirectory() as tmpdir:
    tmpdir = Path(tmpdir)
    
    # Create an outbox for messages
    outbox = tmpdir / "outbox"
    outbox.mkdir()
    
    # Create a new message
    message = sc.SyftMessage.create(
        sender_email=client.my_email or "alice@example.com",
        recipient_email="bob@example.com",
        message_root=outbox,
        message_type="demo"
    )
    
    print(f"Created message: {message.message_id}")
    print(f"From: {message.sender_email}")
    print(f"To: {message.recipient_email}")
    print(f"Timestamp: {message.timestamp}")

### Adding Files to a Message

Messages can contain multiple files with individual permissions:

In [ ]:
with tempfile.TemporaryDirectory() as tmpdir:
    tmpdir = Path(tmpdir)
    
    # Create some test data
    test_file = tmpdir / "data.csv"
    test_file.write_text("id,name,value\n1,Alice,100\n2,Bob,200\n3,Charlie,300")
    
    config_file = tmpdir / "config.json"
    config_file.write_text(json.dumps({"version": "1.0", "debug": False}, indent=2))
    
    # Create message
    outbox = tmpdir / "outbox"
    outbox.mkdir()
    
    message = sc.SyftMessage.create(
        sender_email="alice@example.com",
        recipient_email="bob@example.com",
        message_root=outbox
    )
    
    # Add files with permissions
    file1 = message.add_file(
        source_path=test_file,
        syftbox_path="/alice@example.com/shared/data.csv",
        permissions={
            "read": ["bob@example.com", "charlie@example.com"],
            "write": ["alice@example.com"],
            "admin": ["alice@example.com"]
        }
    )
    
    file2 = message.add_file(
        source_path=config_file,
        syftbox_path="/alice@example.com/config/settings.json",
        permissions={
            "read": ["bob@example.com"],
            "write": [],
            "admin": ["alice@example.com"]
        }
    )
    
    # Add a README
    message.add_readme("""
    <html>
    <body>
        <h1>Data Package</h1>
        <p>This package contains:</p>
        <ul>
            <li>data.csv - Sample dataset</li>
            <li>settings.json - Configuration file</li>
        </ul>
    </body>
    </html>
    """)
    
    # List files in the message
    print("Files in message:")
    for f in message.get_files():
        print(f"\n  {f['filename']}:")
        print(f"    Size: {f['file_size']} bytes")
        print(f"    Path: {f['syftbox_path']}")
        print(f"    Can read: {f['permissions']['read']}")
    
    # Finalize the message
    message.finalize()
    print(f"\nMessage finalized: {message.is_ready}")
    
    # Validate the message
    is_valid, error = message.validate()
    print(f"Message valid: {is_valid}")

## 6. Two-User Workflow Simulation

Let's simulate how two users would set up bidirectional communication. In reality, these would be on different machines.

In [ ]:
# This is a simulation - in practice, each user runs their part separately
print("=== TWO-USER WORKFLOW SIMULATION ===")
print("\nIn a real scenario, Alice and Bob would each run their parts on their own machines.\n")

print("ALICE's Steps:")
print("1. Alice logs in: client = sc.login('alice@gmail.com')")
print("2. Alice creates SyftBox: client.reset_syftbox()")
print("3. Alice adds Bob: client.add_friend('bob@gmail.com')")
print("4. Alice's friends list: ['bob@gmail.com']")

print("\nBOB's Steps:")
print("1. Bob logs in: client = sc.login('bob@gmail.com')")
print("2. Bob creates SyftBox: client.reset_syftbox()")
print("3. Bob checks requests: client.friend_requests -> ['alice@gmail.com']")
print("4. Bob adds Alice back: client.add_friend('alice@gmail.com')")
print("5. Bob's friends list: ['alice@gmail.com']")

print("\n✅ Bidirectional channel established!")
print("Both users can now securely share files through their SyftBoxes.")

## 7. Google Drive Operations

The client also provides direct Google Drive operations:

In [ ]:
# List files in root directory
print("Files in Google Drive root:")
files = client.list_files()

# Show first 5 files
for file in files[:5]:
    file_type = "📁" if file['mimeType'] == 'application/vnd.google-apps.folder' else "📄"
    print(f"{file_type} {file['name']}")

if len(files) > 5:
    print(f"... and {len(files) - 5} more files")

### Creating and Managing Folders

In [ ]:
# Create a test folder
test_folder = client.create_folder("SyftClient_Test_Folder")
print(f"Created folder: {test_folder['name']}")
print(f"Folder ID: {test_folder['id']}")

# Create a subfolder
subfolder = client.create_folder("Subfolder", parent_id=test_folder['id'])
print(f"\nCreated subfolder: {subfolder['name']} in {test_folder['name']}")

# Clean up - delete the test folder
# Uncomment to actually delete
# client.delete_file(test_folder['id'])
# print("\nTest folder deleted")

## 8. Security Features

SyftFileBackedView provides security features to protect against malicious input:

In [ ]:
# Demonstrate path traversal protection
with tempfile.TemporaryDirectory() as tmpdir:
    view = sc.SyftFileBackedView(Path(tmpdir) / "secure_object")
    
    # These dangerous names will be rejected
    dangerous_names = [
        "../../../etc/passwd",
        "./../../sensitive.txt",
        ".hidden_file.txt"  # Hidden files not allowed
    ]
    
    print("Testing path security:")
    for name in dangerous_names:
        try:
            view.write_data_file(name, b"test")
            print(f"  ❌ SECURITY ISSUE: {name} was allowed!")
        except ValueError as e:
            print(f"  ✅ Blocked: {name}")
    
    # These safe names are allowed
    safe_names = ["data.txt", "report.pdf", "results.csv"]
    
    print("\nTesting safe filenames:")
    for name in safe_names:
        try:
            view.write_data_file(name, b"safe content")
            print(f"  ✅ Allowed: {name}")
        except ValueError as e:
            print(f"  ❌ Wrongly blocked: {name}")

## 9. Best Practices

Here are some important tips for using syft-client effectively:

In [ ]:
# Best Practice 1: Always check authentication
def safe_operation():
    client = sc.login("user@gmail.com", verbose=False)
    if not client.authenticated:
        print("❌ Authentication failed")
        return False
    
    print("✅ Authenticated successfully")
    # Proceed with operations
    return True

# Best Practice 2: Handle errors gracefully
def robust_friend_add(client, friend_email):
    try:
        success = client.add_friend(friend_email)
        if success:
            print(f"✅ Added {friend_email}")
        else:
            print(f"⚠️  Could not add {friend_email}")
        return success
    except Exception as e:
        print(f"❌ Error adding friend: {e}")
        return False

# Best Practice 3: Clean up test data
def cleanup_test():
    """Clean up after testing"""
    client = sc.login("test@gmail.com", verbose=False)
    if client.authenticated:
        # Reset removes all SyftBox data
        client.reset_syftbox()
        print("✅ Test data cleaned up")

print("Best practices demonstrated above")

## 10. Troubleshooting

Common issues and their solutions:

In [ ]:
print("🔧 TROUBLESHOOTING GUIDE\n")

print("Issue: 'credentials.json not found'")
print("Solution: Run sc.wizard() to create credentials\n")

print("Issue: 'Token expired'")
print("Solution: Use force_relogin=True")
print("Example: client = sc.login(email, force_relogin=True)\n")

print("Issue: 'Friend not showing up'")
print("Solution: Both users must add each other")
print("Check: client.friend_requests for pending requests\n")

print("Issue: 'SyftBox already exists'")
print("Solution: Use client.reset_syftbox() to start fresh\n")

print("Issue: 'Authentication keeps failing'")
print("Solution: Clear credentials and re-authenticate")
print("Example: sc.logout(email) then sc.login(email)")

## 11. Summary

Congratulations! You've learned how to:

✅ **Authenticate** with Google Drive  
✅ **Create a SyftBox** for secure communication  
✅ **Manage friends** and connection requests  
✅ **Create secure messages** with file attachments  
✅ **Handle permissions** for shared files  
✅ **Use security features** to protect data  
✅ **Follow best practices** for production use  

### Next Steps

1. **Try it with a friend**: Set up a real bidirectional channel
2. **Explore the API**: Check [API_REFERENCE.md](API_REFERENCE.md) for detailed documentation
3. **Build something**: Use syft-client in your own projects
4. **Contribute**: Report issues or contribute at [GitHub](https://github.com/OpenMined/syft-client)

Happy secure file sharing! 🚀

## Appendix: Quick Reference

Here's a quick reference of the most common operations:

In [ ]:
# Quick Reference Card
reference = """
AUTHENTICATION
--------------
sc.login(email)                    # Login
sc.login(email, force_relogin=True) # Force fresh auth
sc.list_accounts()                 # List saved accounts
sc.logout(email)                   # Remove credentials

CLIENT OPERATIONS
-----------------
client.authenticated               # Check auth status
client.my_email                   # Get current email
client.reset_syftbox()            # Create/reset SyftBox

FRIEND MANAGEMENT
-----------------
client.add_friend(email)          # Add a friend
client.friends                    # List friends
client.friend_requests           # Check requests

GOOGLE DRIVE
------------
client.list_files()              # List files
client.create_folder(name)       # Create folder
client.upload_file(path)         # Upload file
client.download_file(id, path)   # Download file
client.delete_file(id)           # Delete file/folder
client.share_file(id, email)     # Share with someone

MESSAGES
--------
SyftMessage.create(sender, recipient, root)
message.add_file(path, syftbox_path, permissions)
message.finalize()
message.validate()
"""

print(reference)